In [6]:
!pip install annoy

  Using cached annoy-1.17.2.tar.gz (647 kB)
  Created wheel for annoy: filename=annoy-1.17.2-cp39-cp39-win_amd64.whl size=52199 sha256=2e68cf16f590cbdd8085ac7ca82062882c0c6debeed24559d0ebbcae0d0814a9
  Stored in directory: c:\users\zenan\appdata\local\pip\cache\wheels\f2\2e\e4\f3ae385c375b87982a2a70055061d4a6330ef4f60817e717e3
Successfully built annoy


In [24]:
import pandas as pd
import numpy as np
import gensim.downloader as api
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from annoy import AnnoyIndex

nltk.download('punkt')
nltk.download('stopwords')

# Load the pre-trained Word2Vec model
word2vec_model = api.load('word2vec-google-news-300')

# Load the dataset
PATH_TO_DATASET = 'complaints.csv'
data = pd.read_csv(PATH_TO_DATASET)
data = data.fillna('')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zenan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zenan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\zenan\AppData\Local\Temp\ipykernel_15308\3792668802.py:19: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(PATH_TO_DATASET)


In [30]:
x = 10000
df = data.sample(n=x, random_state=42)
df.reset_index(drop=True, inplace=True)

test_df = df.copy()
# Combine the selected columns
test_df['combined_text'] = test_df['Product'] + ' ' + test_df['Sub-product'] + ' ' + test_df['Issue'] + ' ' + test_df['Sub-issue'] + ' ' + test_df['Consumer complaint narrative'] + ' ' + test_df['Company public response'] + ' ' + test_df['Company'] + ' ' + test_df['Company response to consumer']

In [31]:
def text_to_vector(text):
    words = word_tokenize(text)
    words = [word for word in words if word not in stopwords.words('english')]
    words = [word for word in words if word in word2vec_model.key_to_index]  # Updated line
    if len(words) == 0:
        return np.zeros(word2vec_model.vector_size)
    word_vectors = [word2vec_model[word] for word in words]
    return np.mean(word_vectors, axis=0)

test_df['text_vector'] = test_df['combined_text'].apply(text_to_vector)

In [32]:
# Build Annoy index
vector_size = word2vec_model.vector_size
annoy_index = AnnoyIndex(vector_size, metric='angular')

for i, vector in enumerate(test_df['text_vector']):
    annoy_index.add_item(i, vector)

annoy_index.build(50)

True

In [33]:
# Find and remove highly similar rows
threshold = 0.99
to_remove = set()

for i in range(len(test_df)):
    if i not in to_remove:
        similar_items = annoy_index.get_nns_by_item(i, 1000, include_distances=True)
        indices, distances = similar_items
        for index, distance in zip(indices, distances):
            if i != index and 1 - distance / 2 > threshold:
                to_remove.add(index)

test_df.drop(list(to_remove), inplace=True)
test_df.reset_index(drop=True, inplace=True)

print(test_df)

     Date received                                            Product  \
0       2021-04-19                                           Mortgage   
1       2020-04-29  Credit reporting, credit repair services, or o...   
2       2015-08-14                                    Debt collection   
3       2020-08-18                        Credit card or prepaid card   
4       2021-02-13                        Credit card or prepaid card   
...            ...                                                ...   
6299    2020-05-03                        Credit card or prepaid card   
6300    2023-01-19  Credit reporting, credit repair services, or o...   
6301    2020-02-11                                    Debt collection   
6302    2023-02-07                        Credit card or prepaid card   
6303    2022-07-30  Credit reporting, credit repair services, or o...   

                                     Sub-product  \
0                                   FHA mortgage   
1                  